In [1]:
import pandas as pd
from calendar import monthrange
from math import ceil
import numpy as np
import datetime as dt

data = pd.read_csv('./data_input_nozerofill_2015.csv', sep=',', delimiter=',')
data_model = data.copy()

#usando apenas as top 100 linhas (linhas com mais exemplos)
top100_linhalist = data_model.linha.value_counts().index[:100]

In [2]:
## Inserir zero rows
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

# linha_list = data_model.linha.unique()


for linha in top100_linhalist:
    currentDataModel_Linha = data_model.loc[data_model.linha == linha]
    mes_max = currentDataModel_Linha.mes.max()
    mes_min = currentDataModel_Linha.mes.min()
    for mes in range(mes_min, mes_max+1):
        currentDataModel_Mes = currentDataModel_Linha.loc[currentDataModel_Linha.mes == mes]
        dia_max = monthrange(2020, mes)[1] 
        for dia in range(1, dia_max+1):
            currentDataModel_Dia = currentDataModel_Mes.loc[currentDataModel_Mes.d_mes == dia]
            
            for hora in range(0, 24):
                currentDataModel_Hora = currentDataModel_Dia[currentDataModel_Dia.hora == hora]
                if currentDataModel_Hora.empty == False:
                    continue
                else:
                    # print('INSERINDO NO MES '+str(mes)+' NO DIA '+str(dia)+ ' NA HORA '+str(hora))
                    #2018-01-01 00:00:00
                    if (mes == 2 and dia > 28):
                        continue
                    dd = dt.datetime.strptime('2015-'+str(mes)+'-'+str(dia)+' '+str(hora)+':00:00', "%Y-%m-%d %H:00:00")
                    new_row = {
                            'linha': linha,
                            'data_hora': dd,
                            'validations_per_hour': 0,
                            'd_semana': dd.weekday(),
                            'hour_sin': np.sin(2 * np.pi * hora/23.0),
                            'hour_cos': np.cos(2 * np.pi * hora/23.0),
                            'hora': hora,
                            'd_mes': dia,
                            'd_ano': dd.timetuple().tm_yday,
                            'mes': mes,
                            'semana_do_mes': week_of_month(dd)
                        }
                    # print(new_row)
                    data_model = data_model.append(new_row, ignore_index=True)

                #insert validation zero
data_model.loc[data_model.mes == 3]

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes
300056,703,2015-03-03 08:00:00,3,1,8.169699e-01,-0.576680,8,3,62,3,1
300057,703,2015-03-03 09:00:00,3,1,6.310879e-01,-0.775711,9,3,62,3,1
301113,1,2015-03-01 00:00:00,19,6,0.000000e+00,1.000000,0,1,60,3,0
301114,1,2015-03-01 01:00:00,6,6,2.697968e-01,0.962917,1,1,60,3,0
301115,1,2015-03-01 02:00:00,2,6,5.195840e-01,0.854419,2,1,60,3,0
...,...,...,...,...,...,...,...,...,...,...,...
2080673,22,2015-03-30 23:00:00,0,0,-2.449294e-16,1.000000,23,30,89,3,6
2080674,22,2015-03-31 00:00:00,0,1,0.000000e+00,1.000000,0,31,90,3,6
2080675,22,2015-03-31 01:00:00,0,1,2.697968e-01,0.962917,1,31,90,3,6
2080676,22,2015-03-31 02:00:00,0,1,5.195840e-01,0.854419,2,31,90,3,6


In [3]:
mes

12

In [4]:
data_model = data_model.sort_values(['linha', 'data_hora'], ascending=[True, True])

In [5]:
data_model.to_csv('./data_input_zerofill_2015.csv', index=False, sep=';')

In [6]:
data_model

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes
1931193,1,2015-01-15 02:00:00,0,3,0.519584,0.854419,2,15,15,1,3
1931194,1,2015-01-25 02:00:00,0,6,0.519584,0.854419,2,25,25,1,4
1931195,1,2015-01-25 06:00:00,0,6,0.997669,-0.068242,6,25,25,1,4
1931196,1,2015-01-25 07:00:00,0,6,0.942261,-0.334880,7,25,25,1,4
1931197,1,2015-01-25 08:00:00,0,6,0.816970,-0.576680,8,25,25,1,4
...,...,...,...,...,...,...,...,...,...,...,...
1785955,999,2015-12-05 09:00:00,18,5,0.631088,-0.775711,9,5,339,12,0
1785956,999,2015-12-05 10:00:00,64,5,0.398401,-0.917211,10,5,339,12,0
1785957,999,2015-12-05 11:00:00,8,5,0.136167,-0.990686,11,5,339,12,0
1785958,999,2015-12-05 12:00:00,43,5,-0.136167,-0.990686,12,5,339,12,0
